In [1]:
import numpy as np
import matplotlib.pyplot as plt
import argparse
from skimage import io, transform

import torch
import torch.nn as nn
import torch.optim as optim
from torch.autograd import Variable
from torchvision.models import vgg19 
from torchsummary import summary

from util import *

In [1]:
def train(output_image, target_style_representation, target_content_representation, feature_maps, model, optimizer, n_steps, a, b):
    if isinstance(optimizer, optim.SGD):
        for iter in range(n_steps):
            # get feature maps of current output image
            _ = model(output_image)
            output_content_representation = get_content_representation(feature_maps.copy())
            output_style_representation = get_style_representation(feature_maps.copy())

            # calculate errors and backprop
            loss = a * content_loss(target_content_representation, output_content_representation) + b * style_loss(target_style_representation, output_style_representation)
            
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
    elif isinstance(optimizer, optim.LBFGS):
        def closure():
            if torch.is_grad_enabled():
                optimizer.zero_grad()

            _ = model(output_image)
            output_content_representation = get_content_representation(feature_maps.copy())
            output_style_representation = get_style_representation(feature_maps.copy())

            loss = a * content_loss(target_content_representation, output_content_representation) + b * style_loss(target_style_representation, output_style_representation)
            if loss.requires_grad:
                loss.backward()
                
            return loss

        for iter in range(n_steps):
            optimizer.step(closure)
    
    return output_image

In [3]:
def main(style_image, content_image, n_steps, device, optimizer='sgd', lr=1e-3, a=0.1, b=1e-3):
    # for saving model later
    style_name = style_image.split('.')[0]
    content_name = content_image.split('.')[0]

    # load images and models
    style_image = read_image('{}'.format(style_image)).transpose([2, 0, 1])
    content_image = torch.Tensor(read_image('{}'.format(content_image)).transpose([2, 0, 1])).unsqueeze(0)
    style_image = torch.Tensor(transform.resize(style_image, content_image.shape))
    content_image = content_image / content_image.max()

    # get white noise image to run gradient descent on for output image
    output_image = torch.rand(content_image.shape, device=device, requires_grad=True)
    # output_image = Variable(content_image.detach().clone(), requires_grad=True)

    model = vgg19(pretrained=True).features # only want feature maps, don't need classifier portion
    model.eval()
    model.to(device)

    # freeze parameters so transfer learning gradient descent just affects images, doesn't retrain network
    for param in model.parameters():
        param.requires_grad = False

    # summary(model, INPUT_SIZE)
    # print(model)

    content_image = content_image.to(device)
    style_image = style_image.to(device)

    # get style/content representations which are static for a given model/image
    feature_maps = set_hooks(model)
    _ = model(content_image)
    target_content_representation = get_content_representation({k: v.detach() for k, v in feature_maps.items()})
    _ = model(style_image)
    target_style_representation = get_style_representation({k: v.detach() for k, v in feature_maps.items()})

    if optimizer == 'sgd':
        optimizer = optim.SGD([output_image], lr=lr, momentum=0.9)
    elif optimizer == 'lbfgs':
        optimizer = optim.LBFGS([output_image], lr=lr)
    output_image = train(output_image, 
                         target_style_representation, 
                         target_content_representation, 
                         feature_maps, 
                         model, 
                         optimizer, 
                         n_steps, 
                         a, b)

    output_image = output_image.cpu().detach().numpy().squeeze()
    output_image = np.transpose(output_image, [1, 2, 0])
    # print(output_image.min(), output_image.max())
    output_image = (output_image - output_image.min()) / (output_image.max() - output_image.min())
    plt.imsave('{}_{}.png'.format(content_name, style_name), output_image)
    plt.imshow(output_image)
    plt.show()

In [ ]:
style_image = 'degas.jpg'
content_image = 'climbing1.jpg'
n_steps = 100
optimizer = 'lbfgs'
lr = 1e-1

device = 'cuda' if torch.cuda.is_available() else 'cpu'

In [4]:
main(style_image, content_image, n_steps, device, optimizer, lr)

NameError: name 'style_image' is not defined